#   Homework 3
## Sentiment analysis using Neural Networks

Total: 50 Points


In this homework we will perform sentiment analysis using a few simple Neural Network based architectures.
For this problem we use the IMDB Large Movie Review Dataset. The dataset contains 25,000 highly polar movie reviews for both train and test dataset, each with 12,500 positive (greater than equal to 7/10 rating) and 12,500 negative reviews(less than equal to 4/10 rating). 

Use "https://keras.io/" for keras documentation. Please use Python 3. GPU is not required but it will help improve the training speed for each problem.

Please save the notebook with your cell outputs. You will not be graded if your outputs are not present below the homework cell. Also note your outputs will be unique since you will be using your the last numbers of your uni as your random seed (In the third cell). Make sure you submit this iPython file, with the saved outputs. The submission format must be 'hw3/hw3.ipynb'. You will not submit any other files. If you do save your model weights, you will not submit them. You will however, make sure your model weights do get saved in the 'weights' folder and can be retrieved from there as well.

Please fill your details below.



Name: Mikael Brunila

Uni: tmb2176

Email: tmb2176@columbia.edu


In [1]:
 # -*- coding: utf-8 -*-

from os import listdir
import os
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense 
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
#we retrieve train and test file names

train_dir = "./aclImdb/train/"
test_dir = "./aclImdb/test/"
tr_review = [re_filename for re_filename in listdir(train_dir)]
te_review = [re_filename for re_filename in listdir(test_dir)]

#we initialize the train and test arrays

tr_X = []
tr_Y = []
te_X = []
te_Y = []

#we arrange the reviews into the train and test arrays 

for review_file in tr_review:
    f_review = open(train_dir+review_file,  errors='ignore', mode="r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    tr_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        tr_Y.append(1)
    else:
        tr_Y.append(0)
        
for review_file in te_review:
    f_review = open(test_dir+review_file,  errors='ignore', mode="r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    te_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        te_Y.append(1)
    else:
        te_Y.append(0)

We will now create the validation set from the train set

use the last 4 numbers of your uni for the seed value seed to ensure all answers remain unique.

In [3]:
#replace 2 (SEED) with the last 4 numbers of your Uni
#Uni: 
SEED = 2
seed_counter = 0
while(1):

    shuffle_combine = list(zip(tr_X, tr_Y))
    random.seed(SEED+seed_counter)
    seed_counter+=1
    random.shuffle(shuffle_combine)

    tr_X, tr_Y = zip(*shuffle_combine)

    val_X = tr_X[:5000]
    val_Y = tr_Y[:5000]

    counter = 0
    for label in val_Y:
        counter+=label

    print (counter)
    print (seed_counter)
    if(counter>2400 and counter <2600):
        tr_X = tr_X[5000:]
        tr_Y = tr_Y[5000:]
        break

2507
1


In [4]:
print("Length of Train review set : " + str(len(tr_X)))
print("Length of Train label set : " + str(len(tr_Y)))
print("Length of Validation review set : " + str(len(val_X)))
print("Length of Validation label set : " + str(len(val_Y)))
print("Length of Test review set : " + str(len(te_X)))
print("Length of Test label set : " + str(len(te_Y)))
print("*****************************************")
print("Some sample Reviews Train sets and their labels")
print(tr_X[0][:150])
print(tr_Y[0])
print(tr_X[1][:150])
print(tr_Y[1])
print(tr_X[2][:150])
print(tr_Y[2])
print(tr_X[3][:150])
print(tr_Y[3])
print(tr_X[4][:150])
print(tr_Y[4])

Length of Train review set : 20000
Length of Train label set : 20000
Length of Validation review set : 5000
Length of Validation label set : 5000
Length of Test review set : 25000
Length of Test label set : 25000
*****************************************
Some sample Reviews Train sets and their labels
THIS REVIEW IS MOSTLY ALL SPOILERS. IF YOU PLAN ON ENJOYING THIS FILM, DON'T READ THIS REVIEW.<br /><br /> That's the problem with kids TV nowadays. I
0
YETI deserves the 8 star rating because it is the one of the greatest bad movies ever made. I saw it at a midnight screening in L.A. and people were r
1
This movie is a little unusual in that it's got a very slim plot and the movie itself is done at a very slow and leisurely pace. While this makes it p
1
I keep watching this movie over and over and over. I have to watch it at least once a week. I am from Africa and looking at that movie taught me some 
1
I do not fail to recognize Haneke's above-average film-making skills. For example, I 

In [5]:
#we collect all the reviews from train validation and test set to generate 
texts = []
texts += tr_X 
texts += te_X 
texts += val_X
len(texts)



#we clip the sentence length to first 250 words. 
MAX_SEQUENCE_LENGTH = 250

#length of vocab, Tokenizer will only use vocab_len most common words
vocab_len = 25000

#we tokenize the texts and convert all the words to tokens
tokenizer = Tokenizer(num_words=vocab_len)
tokenizer.fit_on_texts(texts)

token_tr_X = tokenizer.texts_to_sequences(tr_X)
token_te_X = tokenizer.texts_to_sequences(te_X)
token_val_X = tokenizer.texts_to_sequences(val_X)

#to ensure all reviews have the same length, we pad the smaller reviews with 0, 
#and cut the larger reviews to a max length 
#(we clip from the top, as the end of the reviews generally have a conclusion which provides better features)
x_train = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
x_test = sequence.pad_sequences(token_te_X, maxlen=MAX_SEQUENCE_LENGTH)
x_val = sequence.pad_sequences(token_val_X, maxlen=MAX_SEQUENCE_LENGTH)


#changes the labels to one-hot encoding
y_train = np_utils.to_categorical(tr_Y)
y_test = np_utils.to_categorical(te_Y)
y_val = np_utils.to_categorical(val_Y)


In [6]:
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('X_val shape:', x_val.shape)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_val shape:', y_val.shape)


print("*****************************************")
print("Tokenized Reviews Train sets and their labels")
print(x_train[0][:20])
print(y_train[0])
print()
print(x_train[1][:20])
print(y_train[1])
print()
print(x_train[2][:20])
print(y_train[2])
print()
print(x_train[3][:20])
print(y_train[3])
print()
print(x_train[4][:20])
print(y_train[4])
print()

X_train shape: (20000, 250)
X_test shape: (25000, 250)
X_val shape: (5000, 250)
y_train shape: (20000, 2)
y_test shape: (25000, 2)
y_val shape: (5000, 2)
*****************************************
Tokenized Reviews Train sets and their labels
[ 1990   438 23532   217  1158     4    29     1 18875     2  1094     5
   565    10   185   382   621    18    10    77]
[ 1.  0.]

[   37  1369  4254     2   315  7480   195 12870    82    32  1805  3491
  1942     1   219   394     1  1641     6    27]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]



********************************************

As you can see the reviews have now been transformed into indices to tokenized vocabulary and the labels have been converted to one-hot encoding. We can now go ahead and feed these sequences to Neural Network Models.

********************************************

# Part A

Building your first model (5 Points)

Construct this sequential model using Keras :

![title](img/model1.jpg)

In [11]:
print('Build model...')

## implement model here

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense 
from keras.layers import Activation
from keras.layers import Flatten

model = Sequential()
model.add(Embedding(input_dim = vocab_len, input_length = MAX_SEQUENCE_LENGTH, output_dim = 128))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compille it here according to instructions

model.compile(loss='categorical_crossentropy', 
         optimizer='adam',
         metrics=['accuracy'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_5 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               6400200   
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 9,600,602
Trainable params: 9,600,602
Non-trainable params: 0
___________________________________________________

In [12]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 25s 1ms/step - loss: 0.3974 - acc: 0.8077 - val_loss: 0.2947 - val_acc: 0.8756
Epoch 2/4
20000/20000 [==============================] - 10s 476us/step - loss: 0.0411 - acc: 0.9865 - val_loss: 0.4519 - val_acc: 0.8476
Epoch 3/4
20000/20000 [==============================] - 9s 473us/step - loss: 0.0032 - acc: 0.9992 - val_loss: 0.5624 - val_acc: 0.8610
Epoch 4/4
20000/20000 [==============================] - 9s 474us/step - loss: 1.4947e-04 - acc: 1.0000 - val_loss: 0.6007 - val_acc: 0.8614


# Part B

Stacking Fully Connected Layers (5 points)

Construct this sequential model using Keras :

![title](img/model2.jpg)

In [13]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(input_dim = vocab_len, input_length = MAX_SEQUENCE_LENGTH, output_dim = 128))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compille it here according to instructions

model.compile(loss='categorical_crossentropy', 
         optimizer='adam',
         metrics=['accuracy'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_6 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               6400200   
_________________________________________________________________
activation_3 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
activation_4 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402   

In [14]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 10s 501us/step - loss: 0.4069 - acc: 0.7993 - val_loss: 0.2872 - val_acc: 0.8828
Epoch 2/4
20000/20000 [==============================] - 10s 497us/step - loss: 0.0467 - acc: 0.9847 - val_loss: 0.4460 - val_acc: 0.8470
Epoch 3/4
20000/20000 [==============================] - 10s 488us/step - loss: 0.0062 - acc: 0.9981 - val_loss: 0.7219 - val_acc: 0.8540
Epoch 4/4
20000/20000 [==============================] - 10s 486us/step - loss: 0.0099 - acc: 0.9961 - val_loss: 0.6810 - val_acc: 0.8528


# Part C

Using LSTMS based networks(5 Points) 

Construct this sequential model using Keras :

![title](img/model3.jpg)

In [17]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(input_dim = vocab_len, input_length = MAX_SEQUENCE_LENGTH, output_dim = 128))
model.add(LSTM(128))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compille it here according to instructions

model.compile(loss='categorical_crossentropy', 
         optimizer='adam',
         metrics=['accuracy'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_9 (Activation)    (None, 2)                 0         
Total params: 3,348,354
Trainable params: 3,348,354
Non-trainable params: 0
___________________________________________________

In [16]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 243s 12ms/step - loss: 0.4278 - acc: 0.8003 - val_loss: 0.3225 - val_acc: 0.8686
Epoch 2/5
20000/20000 [==============================] - 244s 12ms/step - loss: 0.2147 - acc: 0.9191 - val_loss: 0.3629 - val_acc: 0.8362
Epoch 3/5
20000/20000 [==============================] - 246s 12ms/step - loss: 0.1180 - acc: 0.9580 - val_loss: 0.3746 - val_acc: 0.8618
Epoch 4/5
20000/20000 [==============================] - 244s 12ms/step - loss: 0.0851 - acc: 0.9701 - val_loss: 0.4347 - val_acc: 0.8436
Epoch 5/5
20000/20000 [==============================] - 242s 12ms/step - loss: 0.1075 - acc: 0.9592 - val_loss: 0.6028 - val_acc: 0.8686


# Part D

Adding Pretrained Word Embeddings(10 Points)

Construct this sequential model using Keras :

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model4.jpg)

In [7]:
import codecs
from keras.layers import LSTM

#dimension of Glove Embeddings.
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

#load glove embeddings
gembeddings_index = {}
with codecs.open('glove.42B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

# nb_words contains the total length of vocab
nb_words = len(word_index) +1

#get glove embeddings for each word in tokenizer.
#g_word_embedding_matrix holds the embeddings dictionary
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
#total words in the tokenizer not in Embedding matrix
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))



Found 122594 unique tokens
G Word embeddings: 1917494
G Null word embeddings: 34289


In [30]:
print('Build model...')

## implement model here

## to use the glove embeddings, your embedding layer would take the vocab size as input dimension, 
## Glove embedding dimension as the output dimsion
## and you will provide the  embedding dictionary as the 'weights' parameter (!important) to the embedding layer.

model = Sequential()
model.add(Embedding(input_dim = nb_words,
                    output_dim = EMBEDDING_DIM, 
                    weights = [g_word_embedding_matrix], 
                    input_length = MAX_SEQUENCE_LENGTH))
model.add(LSTM(128, recurrent_dropout = 0.2))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compille it here according to instructions

model.compile(loss='categorical_crossentropy', 
         optimizer='adam',
         metrics=['accuracy'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 250, 300)          36778500  
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_10 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 258       
_________________________________________________________________
activation_11 (Activation)   (None, 2)                 0     

In [31]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 299s 15ms/step - loss: 0.5048 - acc: 0.7443 - val_loss: 0.3179 - val_acc: 0.8724
Epoch 2/5
20000/20000 [==============================] - 298s 15ms/step - loss: 0.2372 - acc: 0.9123 - val_loss: 0.2618 - val_acc: 0.8920
Epoch 3/5
20000/20000 [==============================] - 296s 15ms/step - loss: 0.1165 - acc: 0.9622 - val_loss: 0.3073 - val_acc: 0.8996
Epoch 4/5
20000/20000 [==============================] - 294s 15ms/step - loss: 0.0516 - acc: 0.9850 - val_loss: 0.3677 - val_acc: 0.8912
Epoch 5/5
20000/20000 [==============================] - 295s 15ms/step - loss: 0.0215 - acc: 0.9941 - val_loss: 0.5258 - val_acc: 0.8838


# Dont attempt this

Stacking LSTM layers

Unfortunately it takes very long to train, be aware we can stack LTMSs over each other like this.
This requires bottom LSTM to return a sequences instead instead of single vector, which becomes input for the top LSTM.


![title](img/model5.jpg)

# Part E

Using Convolutional Networks (10 points)

Construct the model, shown below. Use the same loss functions and optimizers as before

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model6.jpg)

In [34]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(input_dim = nb_words,
                    output_dim = EMBEDDING_DIM, 
                    weights = [g_word_embedding_matrix], 
                    input_length = MAX_SEQUENCE_LENGTH))
model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(64, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(32, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compille it here according to instructions

model.compile(loss='categorical_crossentropy', 
         optimizer='adam',
         metrics=['accuracy'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 250, 300)          36778500  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_18 (Activation)   (None, 248, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_9 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_19 (Activation)   (None, 246, 64)           0     

In [35]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 38s 2ms/step - loss: 0.4508 - acc: 0.7614 - val_loss: 0.2680 - val_acc: 0.8962
Epoch 2/5
20000/20000 [==============================] - 36s 2ms/step - loss: 0.2240 - acc: 0.9145 - val_loss: 0.3380 - val_acc: 0.8592
Epoch 3/5
20000/20000 [==============================] - 36s 2ms/step - loss: 0.1318 - acc: 0.9519 - val_loss: 0.2651 - val_acc: 0.9014
Epoch 4/5
20000/20000 [==============================] - 36s 2ms/step - loss: 0.0752 - acc: 0.9739 - val_loss: 0.3439 - val_acc: 0.8952
Epoch 5/5
20000/20000 [==============================] - 36s 2ms/step - loss: 0.0415 - acc: 0.9856 - val_loss: 0.4033 - val_acc: 0.8934


# Part F

Model constructed : (5 points)

Test Accuracy Over 87.5%: (5 Points)

Bonus: Min(10, Square of (test_score - 88%))

Create your best model, use Validation score to judge your best model and check accuracy on test set


In [23]:
import h5py

print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(input_dim = nb_words,
                    output_dim = EMBEDDING_DIM, 
                    weights = [g_word_embedding_matrix], 
                    input_length = MAX_SEQUENCE_LENGTH))
model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(64, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(32, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

## compille it here according to instructions

model.compile(loss='categorical_crossentropy', 
         optimizer='adam',
         metrics=['accuracy'])
model.summary()

print("Model Built")

model.save_weights("./weights/model_best.h5")

print("Saved model to disk")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 300)          36778500  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_6 (Activation)    (None, 248, 128)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_6 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_7 (Activation)    (None, 246, 64)           0     

You can keep saving models with different names in model_name, 

so you can retrieve their weights again for testing, you dont have to retrain 
(You would have to initialize the model definition again).

In [24]:
wt_dir = "./weights/"
model_name = 'model_best'
early_stopping =EarlyStopping(monitor='val_acc', patience=2)
bst_model_path = wt_dir + model_name + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True)

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=7,
          validation_data=(x_val, y_val),
          verbose = 1,
          shuffle = True,
          callbacks=[early_stopping, model_checkpoint])



Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/7
20000/20000 [==============================] - 36s 2ms/step - loss: 0.4268 - acc: 0.7778 - val_loss: 0.2752 - val_acc: 0.8952
Epoch 2/7
20000/20000 [==============================] - 36s 2ms/step - loss: 0.2221 - acc: 0.9148 - val_loss: 0.2949 - val_acc: 0.8838
Epoch 3/7
20000/20000 [==============================] - 36s 2ms/step - loss: 0.1277 - acc: 0.9539 - val_loss: 0.2951 - val_acc: 0.8908


If you plan on using Ensemble averaging, feel free to edit the code below or add multiple models.

Make sure they get saved and can be retrieved when executing serially.

In [25]:
model.load_weights(bst_model_path)
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 3s 139us/step
Accuracy: 88.40%


# Part G

Explain how Dense, LSTM and Convolution Layers work.

Explain Relu, Dropout, and Softmax work.

Analyze the architectures you constructed, with the accuracies you achieved and the training time it took. 

What are some insights you gained with these experiments? 

(5 Points)
